# An introduction to the Black litterman in python
\section*{The Black--Litterman Formulas}

Assume that we have $N$ assets and $K$ views. There are two sets of inputs to the procedure. 
The first set of inputs relate to market parameters and these are:

\begin{align*}
\mathbf{w} &: \text{ A column vector } (N \times 1) \text{ of equilibrium market weights of the assets} \\
\mathbf{\Sigma} &: \text{ A covariance matrix } (N \times N) \text{ of the assets} \\
R_f &: \text{ The risk-free rate} \\
\delta &: \text{ The investor's risk aversion parameter} \\
\tau &: \text{ A scalar indicating the uncertainty of the prior (details below)}
\end{align*}







\section*{The Master Formula}

The first step of the procedure is a \textit{reverse-optimization} step that infers the implied returns vector $\boldsymbol{\pi}$ that are implied by the equilibrium weights $\mathbf{w}$ using the formula:


$[\Pi = \Sigma \sum w]$

Next, the posterior returns and covariances are obtained from the \textit{Black--Litterman Master Formula}, which is the following set of equations:

\begin{align*}
\boldsymbol{\mu}^{BL} &= 
\left[ (\tau \mathbf{\Sigma})^{-1} + \mathbf{P}^\top \mathbf{\Omega}^{-1} \mathbf{P} \right]^{-1}
\left[ (\tau \mathbf{\Sigma})^{-1} \boldsymbol{\pi} + \mathbf{P}^\top \mathbf{\Omega}^{-1} \mathbf{Q} \right] \\
\\
\mathbf{\Sigma}^{BL} &= 
\mathbf{\Sigma} + 
\left[ (\tau \mathbf{\Sigma})^{-1} + \mathbf{P}^\top \mathbf{\Omega}^{-1} \mathbf{P} \right]^{-1}
\end{align*}


# Implementation


In [1]:
import numpy as np
import pandas as pd

def as_colvec(x):
    if (x.ndim == 2):
        return x
    else:
        return np.expand_dims(x, axis=1)
        

In [16]:
def implied_returns(delta, sigma, w):
    """
Obtain the implied expected returns by reverse engineering the weights
Inputs:
delta: Risk Aversion Coefficient (scalar)
sigma: Variance-Covariance Matrix (N x N) as DataFrame
    w: Portfolio weights (N x 1) as Series
Returns an N x 1 vector of Returns as Series
    """
    ir = delta * sigma.dot(w).squeeze() # to get a series from a 1-column dataframe
    ir.name = 'Implied Returns'
    return ir

# Assumes that Omega is proportional to the variance of the prior
def proportional_prior(sigma, tau, p):
    """
    Returns the He-Litterman simplified Omega
    Inputs:
    sigma: N x N Covariance Matrix as DataFrame
    tau: a scalar
    p: a K x N DataFrame linking Q and Assets
    returns a P x P DataFrame, a Matrix representing Prior Uncertainties
    """
    helit_omega = p.dot(tau * sigma).dot(p.T)
    # Make a diag matrix from the diag elements of Omega
    return pd.DataFrame(np.diag(np.diag(helit_omega.values)),index=p.index, columns=p.index)



In [18]:
from numpy.linalg import inv

def bl(w_prior, sigma_prior, p, q,
                omega=None,
                delta=2.5, tau=.02):
    """
# Computes the posterior expected returns based on 
# the original black litterman reference model
#
# W.prior must be an N x 1 vector of weights, a Series
# Sigma.prior is an N x N covariance matrix, a DataFrame
# P must be a K x N matrix linking Q and the Assets, a DataFrame
# Q must be an K x 1 vector of views, a Series
# Omega must be a K x K matrix a DataFrame, or None
# if Omega is None, we assume it is
#    proportional to variance of the prior
# delta and tau are scalars
    """
    if omega is None:
        omega = proportional_prior(sigma_prior, tau, p)
    # Force w.prior and Q to be column vectors
    # How many assets do we have?
    N = w_prior.shape[0]
    # And how many views?
    K = q.shape[0]
    # First, reverse-engineer the weights to get pi
    pi = implied_returns(delta, sigma_prior,  w_prior)
    # Adjust (scale) Sigma by the uncertainty scaling factor
    sigma_prior_scaled = tau * sigma_prior  
    # posterior estimate of the mean, use the "Master Formula"
    # we use the versions that do not require
    # Omega to be inverted (see previous section)
    # this is easier to read if we use '@' for matrixmult instead of .dot()
    #     mu_bl = pi + sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ (q - p @ pi)
    mu_bl = pi + sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega).dot(q - p.dot(pi).values))
    # posterior estimate of uncertainty of mu.bl
#     sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ p @ sigma_prior_scaled
    sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega)).dot(p).dot(sigma_prior_scaled)
    return (mu_bl, sigma_bl)

### A Simple Example: Absolute Views

We start with a simple 2-Asset example. Let's start with an example from _Statistical Models and Methods for Financial Markets (Springer Texts in Statistics) 2008th Edition, Tze Lai and Haipeng Xing_.

Consider the portfolio consisting of just two stocks: Intel (INTC) and Pfizer (PFE).

From Table 3.1 on page 72 of the book, we obtain the covariance matrix (multipled by $10^4$)

\begin{array}{lcc}
INTC & 46.0 & 1.06 \\
PFE   & 1.06 & 5.33
\end{array}

Assume that Intel has a market capitalization of approximately USD 80B and that of Pfizer is approximately USD 100B (this is not quite accurate, but works just fine as an example!).
Thus, if you held a market-cap weighted portfolio you would hold INTC and PFE with the following weights: $W_{INTC} = 80/180 = 44\%, W_{PFE} = 100/180 = 56\%$. These appear to be reasonable weights without an extreme allocation to either stock, even though Pfizer is slightly overweighted.

We can compute the equilibrium implied returns $\pi$ as follows:


In [19]:
tickers = ["INTC", "PFE"]
s = pd.DataFrame([[46.0,1.06],[1.06,5.33]], index  =tickers, columns=tickers)*10E-4
pi = implied_returns(delta = 2.5,sigma = s,w = pd.Series([.44,.56], index = tickers))
pi

INTC    0.052084
PFE     0.008628
Name: Implied Returns, dtype: float64

In [20]:
# for convenience and readability, define the inverse of a dataframe
def inverse(d):
    """
    Invert the dataframe by inverting the underlying matrix
    """
    return pd.DataFrame(inv(d.values), index=d.columns, columns=d.index)

def w_msr(sigma, mu, scale=True):
    """
    Optimal (Tangent/Max Sharpe Ratio) Portfolio weights
    by using the Markowitz Optimization Procedure
    Mu is the vector of Excess expected Returns
    Sigma must be an N x N matrix as a DataFrame and Mu a column vector as a Series
    This implements page 188 Equation 5.2.28 of
    "The econometrics of financial markets" Campbell, Lo and Mackinlay.
    """
    w = inverse(sigma).dot(mu)
    if scale:
        w = w/sum(w) # fix: this assumes all w is +ve
    return w


In [13]:
mu_exp = pd.Series([0.02, 0.04], index=tickers)
np.round(w_msr(s, mu_exp)*100,2)

INTC     3.41
PFE     96.59
dtype: float64

Now we mix those with the views of the investor

In [21]:
# Absolute view 1: INTC will return 2%
# Absolute view 2: PFE will return 4%
q = pd.Series({'INTC': 0.02, 'PFE': 0.04})

# The Pick Matrix
# For View 2, it is for PFE
p = pd.DataFrame([
# For View 1, this is for INTC
    {'INTC': 1, 'PFE': 0},
# For View 2, it is for PFE
    {'INTC': 0, 'PFE': 1}
    ])

# Find the Black Litterman Expected Returns
bl_mu, bl_sigma = bl(w_prior=pd.Series({'INTC':.44, 'PFE':.56}), sigma_prior=s, p=p, q=q)
# Black Litterman Implied Mu
bl_mu


INTC    0.037622
PFE     0.024111
dtype: float64